INSTALL PACKAGES

In [3]:
!pip -q install langchain

In [4]:
!pip -q install bitsandbytes accelerate transformers

In [5]:
!pip -q install sentence_transformers

In [6]:
!pip -q install unstructured

In [7]:
# !pip install pinecone-client==2.2.4
!pip install pinecone-client==2.2.1

  Using cached pinecone_client-2.2.1-py3-none-any.whl.metadata (7.8 kB)
Using cached pinecone_client-2.2.1-py3-none-any.whl (177 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 5.0.1
    Uninstalling pinecone-client-5.0.1:
      Successfully uninstalled pinecone-client-5.0.1


In [8]:
# !pip install numpy==1.24.4

In [9]:
!pip install -U langchain-community

IMPORT PACKAGES

In [10]:
!pip install --upgrade pinecone-client urllib3

  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 2.2.1
    Uninstalling pinecone-client-2.2.1:
      Successfully uninstalled pinecone-client-2.2.1


In [11]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer,AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

LOAD DATA

In [12]:
# Data and SiteMap
URLs=["https://ineuron.ai/"]
loader = UnstructuredURLLoader(urls=URLs)
data=loader.load()

In [13]:
data

[Document(metadata={'source': 'https://ineuron.ai/'}, page_content='Learning with iNeuron made\n\nTake your career to the next level with industry ready programs, An entire learning ecosystem at your fingertips to make learning fun. Choose from a range of tech programs and make your next big career switch.\n\n55%\n\nAverage Salary Hike\n\n400+\n\nDifferent Courses\n\n10000+\n\nCareer Transitions\n\n400+\n\nHiring Partners\n\nLIVE NOW\n\nSupport System\n\nOur support system is live again, this time it is bigger, better and faster. Experience a tech community like never seen before\n\nOur Courses\n\nSuccess Stories\n\nFresher\n\nAbhisekh Bhuyan\n\nMLOps engineer\n\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\n\nFrom\n\nFresher\n\nTo\n\n79% Increment\n\nSubham Kanungo\n\nAssociate Data Scientist\n\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\n\nFrom\n\nTo\n\n1

CHUNKING

In [14]:
#Chunkings
text_splitter= CharacterTextSplitter(separator='\n',chunk_size=1000,chunk_overlap=200)
text_chunks = text_splitter.split_documents(data)

In [15]:
text_chunks

[Document(metadata={'source': 'https://ineuron.ai/'}, page_content='Learning with iNeuron made\nTake your career to the next level with industry ready programs, An entire learning ecosystem at your fingertips to make learning fun. Choose from a range of tech programs and make your next big career switch.\n55%\nAverage Salary Hike\n400+\nDifferent Courses\n10000+\nCareer Transitions\n400+\nHiring Partners\nLIVE NOW\nSupport System\nOur support system is live again, this time it is bigger, better and faster. Experience a tech community like never seen before\nOur Courses\nSuccess Stories\nFresher\nAbhisekh Bhuyan\nMLOps engineer\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\nFrom\nFresher\nTo\n79% Increment\nSubham Kanungo\nAssociate Data Scientist\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\nFrom\nTo\n100% Increment\nSayan Saha\nSoftware Engineer - Python'),

EMBEDDINGS

In [16]:
#Embedding Model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-V2')
# this model can handle direct texts without needing an explicit tokenizer before to convert the text to tokens. So here, AutoTokenizer is not needed
query_result = embeddings.embed_query("Hello World")

<ipython-input-16-59a29ab7b453>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-V2')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

In [17]:
len(query_result)

384

VECTOR DB FOR STORING EMBEDDINGS

In [18]:
# Create environment for pinecone
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY','pcsk_2EDGsr_73u8oSH1ji98emqhpdfS7cw43K5Pn1cvHvENvUK473FZzTCvzDPm24cMc7TXyxd')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV','us-east-1')


In [19]:
os.environ['PINECONE_API_KEY'] = 'pcsk_2EDGsr_73u8oSH1ji98emqhpdfS7cw43K5Pn1cvHvENvUK473FZzTCvzDPm24cMc7TXyxd'  # Replace with your actual API key
os.environ['PINECONE_API_ENV'] = 'us-east-1'

In [20]:
import pinecone
from pinecone import Pinecone

# Initialize Pinecone using Pinecone class
pinecone = Pinecone(
    api_key=os.environ['PINECONE_API_KEY'] ,
    environment=os.environ['PINECONE_API_ENV']
)

In [23]:
# # import pinecone

# # Initialize Pinecone
# # pinecone.init(api_key='pcsk_2EDGsr_73u8oSH1ji98emqhpdfS7cw43K5Pn1cvHvENvUK473FZzTCvzDPm24cMc7TXyxd', environment='us-east-1-aws')

# # Delete the index - Replace 'index_to_delete' with the actual name of the index you want to delete
# index_to_delete = "website-bot"
# pinecone.delete_index(index_to_delete)

# print(f"Index '{index_to_delete}' deleted successfully.")

In [24]:
# Step 2: Create or Connect to an Index
index_name = "website-bot"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
      name=index_name,
      dimension=384, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec={'serverless': {'cloud': 'aws', 'region': 'us-east-1'}}
      )
index = pinecone.Index(index_name)

In [25]:
index

In [ ]:
# pinecone_client = pinecone.Index(index_name)

In [ ]:
# # import pinecone

# # Initialize Pinecone
# # pinecone.init(api_key='pcsk_2EDGsr_73u8oSH1ji98emqhpdfS7cw43K5Pn1cvHvENvUK473FZzTCvzDPm24cMc7TXyxd', environment='us-east-1-aws')

# # Delete the index - Replace 'index_to_delete' with the actual name of the index you want to delete
# index_to_delete = "website-bot"
# pinecone.delete_index(index_to_delete)

# print(f"Index '{index_to_delete}' deleted successfully.")

In [ ]:
# index_name = "website-bot"

In [26]:
from langchain.vectorstores import Pinecone as LangchainPinecone

In [27]:
docsearch = LangchainPinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

LLM WRAPPER

In [28]:
# Log in to Hugging Face (only needed once per session/notebook)
notebook_login()


In [29]:
## LLM WRAPPER
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

Device set to use cuda:0


In [31]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

<ipython-input-31-9753c688c802>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [32]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

<ipython-input-32-181e585c8cae>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide a concise summary of the Book Harry Potter")


"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone by J.K. Rowling.\n\nHarry Potter is an orphan who lives with his cruel and neglectful relatives, the Dursleys. On his eleventh birthday, he discovers that he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. He makes two best friends, Ron Weasley and Hermione Granger, and together they become entangled in a mystery surrounding the powerful Sorcerer's Stone.\n\nThe trio must prevent the stone from falling into the wrong hands, as it has the power to grant eternal life and grant wishes. Along the way, they encounter obstacles such as the school's potions master, Severus Snape, and the school's rival, Draco Malfoy.\n\nThe story is set in a magical world where wizards and witches live secretly among non-magical people, and where magic is a powerful force that can be used for good or evil. The book is full of adventure, humor, and heart, and introduces readers to a richly imagin

RETRIEVAL QA


In [33]:
from langchain.chains import RetrievalQA

In [34]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [35]:
query = "Tell me the course price of Full Stack Data Science with Generative AI provide by ineuron"


In [36]:
print(qa.run(query))

<ipython-input-36-260aa8d7cbf0>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa.run(query))


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Neuro Lab
Get premium access to a state-of-the-art virtual lab with infinite computing so you won't need additional investments in high-end PCs
Job Portal
New-age jobs need new-age technology, Build resumes in minutes, Apply for exclusive jobs or hire fresh talent we have you covered.
One Neuron
Specialized bundled programs tailored to cater to your specific requirements. 500+ Tech courses bundled to make learning easy and affordable
Support System
Complex doubt or joining a like minded community at your fingertips
Internship Portal
Choose from over 500+ live projects across various domains Build, collaborate and grow with peers.
TODO
Completed
In Progress
Backlog
Become an affiliate
Earn while you learn
Hall of fame
Learn from alumnus who cracked the code
I want to express my gratitude to Krish Naik, Sudhanshu Kumar and Sun

In [42]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [44]:
question = "Do they teach Machine Learning in Ineuron?"
result =qa_chain.run(question)
# Check the result of the query
print(result)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
I want to express my gratitude to Krish Naik, Sudhanshu Kumar and Sunny Savita for their instrumental role in my data science journey.
Abhinav Dwivedi
Data Scientist, Softsensor.ai
Thanks to the iNeuron team for teaching me the skillsets to get this fantastic job. it was an amazing journey of 8-9 months. Special thanks to Sudhanshu sir
Saransh Sehrawat
Associate Consultant, EY
A big thanks to the iNeuron team for helping us to enhance our technical knowledge to achieve everyone's dream. My gratitude to all instructors for all you have.
Payel Tarafder
System Engineer, TCS
It was through Krish's videos that I first learned about INEURON, and without hesitation, I joined the inaugural FULL STACK DATA AN

In [45]:
# # Check the source document from where we
# result["source_documents"][0]